In [ ]:
import pandas as pd
import os


# 処理対象のフォルダパス (Windowsパスはr'...'でRaw Stringにすると安全)
PROJECT_PATH = r'C:\Users\robek\OneDrive\デスクトップ\windows編'
# 出力するExcelファイル名
OUTPUT_FILE = 'Project_Document_Breakdown.xlsx'

# ----------------------------------------------------------------------
# 2. generate_visual_tree (データ生成関数)
# ----------------------------------------------------------------------

def generate_visual_tree(startpath):
    """
    指定されたパスのフォルダ構造をスキャンし、
    ツリー表示用の文字列リストを生成して返す関数
    """
    #Excelシート分割用のデータ収集辞書を初期化
    folder_data = {}
    # インデント用の文字列
    space = '    '
    pipe_space = '│' + space
    #パスが存在しない場合は処理を中断し、エラーメッセージを返す
    if not os.path.isdir(startpath):
        return{'エラー':[f'指定されたフォルダが見つからないか、フォルダではありません。スキップします。']}
    #ルートフォルダの文字列を生成
    root_key = os.path.basename(startpath) + os.sep
    #ルートフォルダ配下のツリー表示用のリストの初期化
    root_lines = []
    #ルートフォルダのパスを一行目に追加
    root_lines.append(root_key)

    #ルートフォルダ直下のファイル・ディレクトリを取得
    root_items = os.listdir(startpath)
    #フルパスからファイルのみ取得、リスト化
    root_files = [
        item for item in root_items
        if os.path.isfile(os.path.join(startpath, item))
    ]
    #ファイル名でソート
    root_files.sort()
    #ルートフォルダ直下にサブディレクトリが存在するか確認
    has_subdirs = any(os.path.isdir(os.path.join(startpath, item)) for item in root_items)
    #ルートフォルダ直下のファイルにインデックスを付与してツリー表示用の文字列を生成
    for i, file in enumerate(root_files):
        # ルートフォルダ直下の最後のファイルかどうかを判定(True/False)
        is_last_file = (i == len(root_files) - 1)
        #ルートフォルダの直下にサブフォルダがない、かつ最後のファイルの場合は'└── 'を使用
        if is_last_file and not has_subdirs:
            root_prefix = '└── '
        #それ以外の場合は'├── 'を使用
        else:
            root_prefix = '├── '
        #ツリー表示用の文字列を生成してリストに追加
        root_lines.append(f'{root_prefix}📄{file}')
    #ルートフォルダのデータを辞書に追加(Excelのルートシート用)
    folder_data[root_key] = root_lines
    #os.walkでサブフォルダ以下の全階層を巡回
    for root, dirs, files in os.walk(startpath):
        # ルートフォルダの場合、サブフォルダ名を辞書に追加してスキップ
        if root == startpath:
            for dir_name in dirs:
                #リストを空で初期化する
                folder_data[dir_name + os.sep] = []
            continue
        #ディレクトリ・ファイルをアルファベット順にソート
        dirs.sort()
        files.sort()

        #ルートフォルダからの相対パスを取得
        root_relative = root.replace(startpath + os.sep, '')
        #Excelシートのキーとなるトップレベルディレクトリ名の取得
        top_level_dir = root_relative.split(os.sep)[0] + os.sep
        #トップレベルフォルダが有効なシートキーとして存在するか確認
        if top_level_dir in folder_data:
            current_tree_list = folder_data[top_level_dir]
        else:
            continue
        #ルートからの相対的な階層の深さ(区切り文字数)を計算
        level = root_relative.count(os.sep)
        #追記
        is_top_level_dir = (level == 1)

        if not is_top_level_dir:
            #Excelシートの起点に合わせ、階層レベルを調整(‐１)してインデントを計算
            indent_base = pipe_space * (level - 1)
            #現在処理中のフォルダ名(パス区切り文字を含まない)を取得
            dir_name = os.path.basename(root)
            #(追記)
            if level == 1:
                continue
            #現在のフォルダが親フォルダ内にある最後のフォルダかどうかを判定
            is_last_dir_in_list = dir_name == (dirs[-1] if dirs else dir_name)
            #最後のフォルダの有無でインデントと接続文字を設定
            if is_last_dir_in_list:
                folder_prefix = '└── '
                next_indent = space
            else:
                folder_prefix = '├── '
                next_indent = pipe_space
            #現在のフォルダ名をツリー表示用のリストに追加
            current_tree_list.append(f'{indent_base}{folder_prefix}📂{dir_name}{os.sep}')
        
        file_indent_base = indent_base + next_indent
        #ファイルを順番に処理
        for i, file in enumerate(files):
            #ファイルがリストの中で最後の要素かどうかを判定
            is_last_file = (i == len(files) - 1)
            #最後のファイルかどうかでfile_prefix(接続文字)を決定
            if is_last_file:
                file_prefix = '└── '
            else:
                file_prefix = '├── '
            #ファイル名をツリー表示用のリストに追加
            current_tree_list.append(f'{file_indent_base}{file_prefix}📄{file}')

    return folder_data

# ----------------------------------------------------------------------
# 3. write_trees_to_excel (Excel出力関数)
# ----------------------------------------------------------------------

def write_trees_to_excel(folder_data, output_filepath):
    
    print(f'\n📊 Excelファイルへの書き出しを開始します: {output_filepath}')
    
    try:
        with pd.ExcelWriter(output_filepath, engine='xlsxwriter') as writer:
            
            for folder_name, tree_lines in folder_data.items():
                
                if folder_name == 'エラー':
                    print(f"   警告：データにエラー情報が含まれていました。スキップします: {tree_lines[0]}")
                    continue
                    
                df = pd.DataFrame(tree_lines, columns=['Path']) 
                
                sheet_name = folder_name.replace(os.sep, '_').replace('/', '_').replace('\\', '_')
                sheet_name = sheet_name.strip('_')
                
                if not sheet_name: 
                    sheet_name = 'ルート'
                
                sheet_name = sheet_name[:31] 
                
                df.to_excel(writer, sheet_name=sheet_name, index=False)
                
                try:
                    worksheet = writer.sheets[sheet_name]
                    worksheet.set_column(0, 0, 50) 
                except Exception as e:
                    pass

        print(f"✅ Excelファイルが正常に作成されました。")

    except Exception as e:
        print(f"❌ Excelファイル書き出し中に致命的なエラーが発生しました: {e}")

# ----------------------------------------------------------------------
# 4. メイン処理の実行
# ----------------------------------------------------------------------

if __name__ == "__main__":
    # 現場の例え：施工管理。まずデータを集め、次に帳簿に出力する。
    
    print("--- 建築DXツール：フォルダ構造解析開始 ---")
    
    # データを生成
    tree_data = generate_visual_tree(PROJECT_PATH)
    
    # エラーチェック
    if 'エラー' in tree_data:
        print(f"🚨 処理中断：{tree_data['エラー'][0]}")
    else:
        # Excelに出力
        write_trees_to_excel(tree_data, OUTPUT_FILE)
    
    print("--- 処理完了 ---")

--- 建築DXツール：フォルダ構造解析開始 ---

📊 Excelファイルへの書き出しを開始します: Project_Document_Breakdown.xlsx
❌ Excelファイル書き出し中に致命的なエラーが発生しました: [Errno 13] Permission denied: 'Project_Document_Breakdown.xlsx'
--- 処理完了 ---
